In [17]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

def download_isone_consolidated_lmp(start_date: str, end_date: str, save_path: str = 'isone_lmp_data') -> pd.DataFrame:
    os.makedirs(save_path, exist_ok=True)
    all_data = []

    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y%m%d')
        filename = f"WW_DALMP_ISO_{date_str}.csv"
        url = f"https://www.iso-ne.com/static-transform/csv/histRpts/da-lmp/{filename}"
        local_path = os.path.join(save_path, filename)

        print(f"Downloading {filename}...")

        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(local_path, 'wb') as f:
                f.write(response.content)

           # Skip first 4 metadata lines so that the 5th becomes header
            df = pd.read_csv(local_path, skiprows=4, header=0)

            # Drop the next line (now the first row), which contains type info
            df = df.iloc[1:].reset_index(drop=True)

            df['DATE'] = current_dt.strftime('%Y-%m-%d')
            all_data.append(df)

        except requests.exceptions.HTTPError:
            print(f"File not found for {date_str} — skipping.")
        except Exception as e:
            print(f"Error processing {date_str}: {e}")

        current_dt += timedelta(days=1)

    if all_data:
        full_df = pd.concat(all_data, ignore_index=True)
        print(f"\nTotal records collected: {len(full_df)}")
        return full_df
    else:
        print("No data was downloaded.")
        return pd.DataFrame()


# Example usage:
if __name__ == "__main__":
    df = download_isone_consolidated_lmp("2024-01-01", "2024-01-05")
    df.to_csv("isone_dalmp_scraped.csv", index=False)
    print(df.head())




Total records collected: 145925
   H        Date Hour Ending Location ID        Location Name Location Type  \
0  D  01/01/2024          01         321  UN.FRNKLNSQ13.810CC  NETWORK NODE   
1  D  01/01/2024          01         322  UN.FRNKLNSQ13.811CC  NETWORK NODE   
2  D  01/01/2024          01         323   UN.FRNKLNSQ13.89CC  NETWORK NODE   
3  D  01/01/2024          01         326  UN.ALTRESCO13.8ALTR  NETWORK NODE   
4  D  01/01/2024          01         327  UN.EDDY    34.5AMOS  NETWORK NODE   

  Locational Marginal Price Energy Component Congestion Component  \
0                     28.44            28.21                  0.0   
1                     28.44            28.21                  0.0   
2                     28.44            28.21                  0.0   
3                     27.28            28.21                  0.0   
4                     28.51            28.21                  0.0   

  Marginal Loss Component        DATE  
0                    0.23  2024-01-01

In [22]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

def download_isone_consolidated_lmp_by_quarter(start_date: str, end_date: str, save_path: str = 'isone_lmp_data') -> None:
    """
    Downloads ISO-NE Day-Ahead LMP CSVs for each day in the date range,
    cleans and combines the data, then saves one CSV per quarter.
    """

    os.makedirs(save_path, exist_ok=True)
    all_data = []

    # Parse date range
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y%m%d')
        filename = f"WW_DALMP_ISO_{date_str}.csv"
        url = f"https://www.iso-ne.com/static-transform/csv/histRpts/da-lmp/{filename}"
        local_path = os.path.join(save_path, filename)

        print(f"Downloading {filename}...")

        try:
            response = requests.get(url)
            response.raise_for_status()

            # Save raw CSV
            with open(local_path, 'wb') as f:
                f.write(response.content)
            # Read and clean: use line 5 as header, skip the next line
            df = pd.read_csv(local_path, skiprows=4, header=0)
            df = df.iloc[1:].reset_index(drop=True)  # remove row with data types

            # Track fetch date separately if needed
            df['Fetched_Date'] = current_dt.strftime('%Y-%m-%d')
            all_data.append(df)

        except requests.exceptions.HTTPError:
            print(f"File not found for {date_str} — skipping.")
        except Exception as e:
            print(f"Error processing {date_str}: {e}")

        current_dt += timedelta(days=1)

    if not all_data:
        print("No data was downloaded.")
        return

    # Combine all downloaded data
    full_df = pd.concat(all_data, ignore_index=True)
    # Convert date column from CSVs (not the fetched date)
    full_df['Date'] = pd.to_datetime(full_df['Date'], format='mixed', errors='coerce')
    # Drop rows where the date could not be parsed
    full_df = full_df.dropna(subset=['Date'])
    # Assign each row to a calendar quarter
    full_df['Quarter'] = full_df['Date'].dt.to_period('Q')

    # Save one CSV file per quarter
    for quarter, group in full_df.groupby('Quarter'):
        quarter_str = f"{quarter}".replace('/', '')  # e.g., '2024Q1'
        output_filename = f"isone_lmp_{quarter_str}.csv"
        output_path = os.path.join(save_path, output_filename)

        group.drop(columns='Quarter').to_csv(output_path, index=False)
        print(f"✅ Saved {output_filename} with {len(group)} records.")


if __name__ == "__main__":
    download_isone_consolidated_lmp_by_quarter("2024-04-01", "2025-03-31")


✅ Saved isone_lmp_2024Q2.csv with 2652984 records.
✅ Saved isone_lmp_2024Q3.csv with 2669472 records.
✅ Saved isone_lmp_2024Q4.csv with 2679325 records.
✅ Saved isone_lmp_2025Q1.csv with 2622822 records.
